In [8]:
import matplotlib.pyplot as plt
import sys
sys.path.append('../app')
from matplotlib.backends.backend_pdf import PdfPages
import preprocess as pp
from data_input import load_buildings_from_osm, load_roads_from_osm
import metrics


In [9]:
place = 'Kadima Zoran, Israel'

local_crs = 'EPSG:2039'
network_type = 'drive'
verbose = True

streets = load_roads_from_osm(place, network_type=network_type)
streets, junctions = pp.get_streets(streets=streets, local_crs=local_crs, get_juncions=True)
junctions, streets = metrics.generate_junctions_metrics(streets)
if verbose: print('Got junction metrics')

if verbose: print('Got streets')

buildings = load_buildings_from_osm(place)
buildings = pp.get_buildings(buildings=buildings, streets=streets, junctions=junctions, local_crs=local_crs, )
if verbose: print('Got buildings')


tessellations = pp.get_tessellation(buildings=buildings, streets=streets, 
                                        tess_mode='morphometric', clim='adaptive')
if verbose: print('Got tessellations')

metrics.generate_building_metrics(buildings)
if verbose: print('Got building metrics')

queen_1 = metrics.generate_tessellation_metrics(tessellations, buildings)
if verbose: print('Got tessellation metrics')

metrics.generate_streets_metrics(streets)
if verbose: print('Got street metrics')

queen_3 = metrics.generate_graph_building_metrics(buildings, streets, queen_1)
if verbose: print('Got graph building metrics')

Got junction metrics
Got streets
No building heights in data
Got buildings


/Users/dpelleg/miniconda3/envs/dssg/lib/python3.12/site-packages/libpysal/graph/base.py:1375: UserWarning: The numba package is used extensively in this module to accelerate the computation of graphs. Without numba, these computations may become unduly slow on large data.
  head, tail, weights = _gabriel(
/Users/dpelleg/work/dssg/DSSG_mapping_Jerus/Notebooks/../app/preprocess.py:164: UserWarning: Tessellation contains MultiPolygon elements. Initial objects should  be edited. Index of affected elements: [156, 175, 1039, 1265, 1694, 2279, 2356, 2377, 2379, 2421, 2460, 2474, 2610, 2615, 3011, 3021, 3029, 3043].
  collapsed, _ = momepy.verify_tessellation(tessellations, buildings)
/Users/dpelleg/miniconda3/envs/dssg/lib/python3.12/site-packages/momepy/functional/_shape.py:890: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cosine_angle)
/Users/dpelleg/miniconda3/envs/dssg/lib/python3.12/site-packages/momepy/functional/_shape.py:890: RuntimeWarning: invalid value e

Got tessellations
Got building metrics


/Users/dpelleg/work/dssg/DSSG_mapping_Jerus/Notebooks/../app/metrics.py:114: UserWarning: The numba package is used extensively in this module to accelerate the computation of graphs. Without numba, these computations may become unduly slow on large data.
  tessellations['tess_covered_area'] = queen_1.describe(tessellations.area)['sum']


Got tessellation metrics
Got street metrics


/Users/dpelleg/miniconda3/envs/dssg/lib/python3.12/site-packages/momepy/functional/_distribution.py:382: RuntimeWarning: invalid value encountered in scalar divide
  mean_distances[i] = sub_matrix.sum() / sub_matrix.nnz
/Users/dpelleg/miniconda3/envs/dssg/lib/python3.12/site-packages/momepy/functional/_distribution.py:285: UserWarning: The indices of the two GeoSeries are different.
  (geoms.distance(geometry.geometry, align=True)).groupby(level=0).mean()


Got graph building metrics



buildings["uID"] = range(len(buildings))
buildings["shared_walls"] = momepy.SharedWallsRatio(buildings).series
buildings["area"] = buildings.area
buildings['eri'] = momepy.EquivalentRectangularIndex(buildings).series
buildings['elongation'] = momepy.Elongation(buildings).series
#buildings['adjacency'] = momepy.BuildingAdjacency(buildings, 'uID').series
buildings['orientation'] = momepy.Orientation(buildings,).series
delaunay = graph.Graph.build_triangulation(
    buildings.centroid, coplanar="clique"
).assign_self_weight()

buildings['alignment'] = momepy.Alignment(buildings,
                                            delaunay, unique_id='uID', orientations='orientation', verbose=False)                      


In [10]:

def create_layer_features_pdf(place, layer, features, layer_type='building'):
    pdf_filename = f'{place}_{layer_type}_features.pdf'
    with PdfPages(pdf_filename) as pdf:
        for feature in features:
            print(feature)
            fig, ax = plt.subplots(figsize=(12, 8))
            layer.plot(feature, ax=ax, scheme="natural_breaks", legend=True)
            ax.set_title(f"{place} - {layer_type.capitalize()} {feature.capitalize()}")
            ax.set_axis_off()
            pdf.savefig(fig)
            plt.close(fig)
    return pdf_filename

In [11]:
buildings_features= ['Area', 'corners',
       'squareness', 'perimeter', 'circular_compactness',
       'square_compactness', 'weighted_axis_compactness', 'courtyard_area',
       'fractal_dimension', 'facade_ratio', 'orientation',
       'longest_axis_length', 'equivalent_rectangular_index', 'elongation',
       'rectangularity', 'shared_walls_length', 'perimeter_wall', 'alignment',
       'adjacency', 'mean_interbuilding_distance', 'neighbour_distance',
       'courtyards_num', 'street_alignment']

In [12]:
create_layer_features_pdf(place, buildings, buildings_features, 'building')

Area
corners
squareness
perimeter
circular_compactness
square_compactness
weighted_axis_compactness
courtyard_area


/Users/dpelleg/miniconda3/envs/dssg/lib/python3.12/site-packages/mapclassify/classifiers.py:687: UserWarning: Not enough unique values in array to form 5 classes. Setting k to 2.
  self._classify()


fractal_dimension
facade_ratio
orientation
longest_axis_length
equivalent_rectangular_index
elongation
rectangularity
shared_walls_length
perimeter_wall


/Users/dpelleg/miniconda3/envs/dssg/lib/python3.12/site-packages/mapclassify/classifiers.py:687: UserWarning: Not enough unique values in array to form 5 classes. Setting k to 3.
  self._classify()


alignment
adjacency
mean_interbuilding_distance
neighbour_distance
courtyards_num
street_alignment


/Users/dpelleg/miniconda3/envs/dssg/lib/python3.12/site-packages/mapclassify/classifiers.py:687: UserWarning: Not enough unique values in array to form 5 classes. Setting k to 2.
  self._classify()


'Kadima Zoran, Israel_building_features.pdf'

In [13]:
streets_features = ['mm_len', 'node_start', 'node_end', 'str_orientation',
       'str_longest_axis', 'str_linearity', 'str_length', 'width', 'openness',
       'width_dev']
create_layer_features_pdf(place, streets, streets_features, 'street')


mm_len
node_start
node_end
str_orientation
str_longest_axis
str_linearity
str_length
width
openness
width_dev


'Kadima Zoran, Israel_street_features.pdf'

In [14]:
tessellations_features = ['tess_area', 'tess_circ_compactness',
       'tess_convexity', 'tess_neighbors', 'tess_covered_area',
       'tess_build_area_raio', 'tess_orientation', 'tess_building_orientation',
       'tess_alignment', 'tess_equivalent_rect_index']
create_layer_features_pdf(place, tessellations, tessellations_features, 'tesselation')


tess_area
tess_circ_compactness
tess_convexity
tess_neighbors
tess_covered_area
tess_build_area_raio
tess_orientation
tess_building_orientation
tess_alignment
tess_equivalent_rect_index


'Kadima Zoran, Israel_tesselation_features.pdf'